In [27]:
from joblib import dump, load
import pandas as pd
import numpy as np
from class_def import *

driver_history = load('driver_history')

In [34]:
#need to convert the list of movements (time, location, passenger)
#to animations (point a to point b movements)
#each animation needs a movement vector, and also a number of frames stopping point
#as well as a starting position and ending position
driver_history[2].movement_history[:5]

[[0, 3, None],
 [93.05980895204506, 183, <class_def.Passenger at 0x16247ddf0>],
 [100.41459750492768, 183, None],
 [188.43322756865277, 208, <class_def.Passenger at 0x16247dc40>],
 [199.24849784984474, 208, None]]

In [35]:
driver_history[2].movement_history[1][2].departure_time

100.41459750492768

In [38]:
driver_history[4].movement_history

[[0, 3, None],
 [64.49351385886686, 18, <class_def.Passenger at 0x162449400>],
 [79.64437571954895, 18, None],
 [145.3654598633153, 119, <class_def.Passenger at 0x162449250>],
 [159.1977599933724, 119, None],
 [159.23257890677544, 116, <class_def.Passenger at 0x1624490a0>],
 [160.58828175399194, 116, None],
 [477.9187256488144, 74, <class_def.Passenger at 0x162444f10>],
 [479.8592605819846, 74, None],
 [480.21601503137686, 74, <class_def.Passenger at 0x162444d60>],
 [480.3414053074781, 74, None],
 [480.4458624169016, 147, <class_def.Passenger at 0x162444bb0>],
 [480.5053268275147, 147, None],
 [480.6694868870957, 69, <class_def.Passenger at 0x162444a60>],
 [480.765820038673, 69, None],
 [481.7848582726896, 138, <class_def.Passenger at 0x1624448e0>],
 [508.7200068514353, 138, None],
 [508.7200068514353, 66, <class_def.Passenger at 0x1624447c0>],
 [538.3116433242352, 66, None],
 [538.3116433242352, 138, None],
 [541.154781233991, 138, None],
 [541.154781233991, 66, <class_def.Passenger a

In [44]:
driver_history[4].movement_history[1::2]

[[64.49351385886686, 18, <class_def.Passenger at 0x162449400>],
 [145.3654598633153, 119, <class_def.Passenger at 0x162449250>],
 [159.23257890677544, 116, <class_def.Passenger at 0x1624490a0>],
 [477.9187256488144, 74, <class_def.Passenger at 0x162444f10>],
 [480.21601503137686, 74, <class_def.Passenger at 0x162444d60>],
 [480.4458624169016, 147, <class_def.Passenger at 0x162444bb0>],
 [480.6694868870957, 69, <class_def.Passenger at 0x162444a60>],
 [481.7848582726896, 138, <class_def.Passenger at 0x1624448e0>],
 [508.7200068514353, 66, <class_def.Passenger at 0x1624447c0>],
 [538.3116433242352, 138, None],
 [541.154781233991, 66, <class_def.Passenger at 0x1624445b0>],
 [564.5736529188889, 49, <class_def.Passenger at 0x1624443d0>],
 [572.5775569470015, 17, <class_def.Passenger at 0x1624442e0>],
 [574.2520977095608, 79, <class_def.Passenger at 0x1624440d0>],
 [626.5648164129703, 158, <class_def.Passenger at 0x16243dee0>],
 [896.9402713569766, 231, <class_def.Passenger at 0x16243dd90>]]

In [45]:
driver_history[4].movement_history[2::2]

[[79.64437571954895, 18, None],
 [159.1977599933724, 119, None],
 [160.58828175399194, 116, None],
 [479.8592605819846, 74, None],
 [480.3414053074781, 74, None],
 [480.5053268275147, 147, None],
 [480.765820038673, 69, None],
 [508.7200068514353, 138, None],
 [538.3116433242352, 66, None],
 [541.154781233991, 138, None],
 [562.5455301973242, 66, None],
 [572.5775569470015, 49, None],
 [574.0136695042077, 17, None],
 [626.3174744942407, 79, None],
 [642.1016423415105, 158, None],
 [898.3698546833252, 231, None]]

In [48]:
ex_driver = driver_history[4].movement_history
saved_location = ex_driver[0][1]
for d, a in zip(ex_driver[1::2], ex_driver[2::2]):
    print(d[0], a[0] - d[0], saved_location, d[1])
    saved_location = d[1]

64.49351385886686 15.150861860682085 3 18
145.3654598633153 13.832300130057092 18 119
159.23257890677544 1.3557028472164916 119 116
477.9187256488144 1.94053493317017 116 74
480.21601503137686 0.12539027610125686 74 74
480.4458624169016 0.0594644106130886 74 147
480.6694868870957 0.09633315157731204 147 69
481.7848582726896 26.935148578745725 69 138
508.7200068514353 29.591636472799905 138 66
538.3116433242352 2.8431379097557965 66 138
541.154781233991 21.39074896333318 138 66
564.5736529188889 8.003904028112629 66 49
572.5775569470015 1.4361125572061155 49 17
574.2520977095608 52.065376784679984 17 79
626.5648164129703 15.536825928540111 79 158
896.9402713569766 1.4295833263486202 158 231
